In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import sys, os
if os.path.abspath('../') not in sys.path:
    sys.path.append(os.path.abspath('../'))

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import matplotlib.pyplot as plt
import numpy as np

In [ ]:
from utils.dummy import DummyDatasetCifar10, DummyModelCifar10

In [ ]:
BATCH_SIZE = 10

In [ ]:
model = DummyModelCifar10()
cifar10 = DummyDatasetCifar10(batch_size=BATCH_SIZE, data_root='../data')

In [ ]:
model.load_state_dict(torch.load('data/dummy.weights'))

In [ ]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)

In [ ]:
%%time
for epoch in range(10):

    running_loss = 0.0
    for i, data in enumerate(cifar10.trainloader, 0):
        # get the inputs; data is a list of [inputs, labels]
        inputs, labels = data

        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        # print statistics
        running_loss += loss.item()
        if i % 2000 == 1999:    # print every 2000 mini-batches
            print('[Epoch %d, Batch %5d] loss: %.3f' %
                  (epoch + 1, i + 1, running_loss / 2000))
            running_loss = 0.0

print('Finished Training')

In [ ]:
model.eval()

In [ ]:
dataiter = iter(cifar10.testloader)
images, labels = dataiter.next()
_, predicted = torch.max(model(images), 1)

fig, axs = plt.subplots(2, 5, figsize=(15, 5))
for ax, img, lbl, pred in zip(axs.flatten(), images, labels, predicted):
    ax.imshow(np.transpose((img / 2 + 0.5).numpy(), (1,2,0)))
    ax.axis('off')
    ax.set_title(f'{cifar10.classes[lbl]}/{cifar10.classes[pred]}')
    
plt.show()

In [ ]:
correct = 0
all_ = len(cifar10.testloader) * BATCH_SIZE
for i, data in enumerate(cifar10.testloader, 0):
        # get the inputs; data is a list of [inputs, labels]
        inputs, labels = data
        _, predicted = torch.max(model(inputs), 1)
        correct += (labels == predicted).sum().detach().numpy()
        
print(f'accuracy: {correct / all_}')

In [ ]:
# torch.save(model.state_dict(), 'data/dummy.weights')